# Welcome to Lexeton - The car speed controller

### Our lives are deeply engrossed in finishing tasks, researching, household chores,etc. Usually we need some materialistic things for fulfilling our needs. One of such items is a car. A car is a daily necessity for the transportation of goods,capital and people. Being widely under use, it has several advantages like quickening our lives, but can also have several disadvantages like accidents leading to death. To reduce such car accidents on highways, city roads,etc. We have built an amazing AI project that will deeply analyze the speed of cars and if found too high then it will to reported to the police. 

___________________________________________________________

### In terms of functioning, first a video can  be recorded by the police which may be fed into the system to detect the speed in the case of which the police may understand to take the necessary actions. However if it is a live system then the email can be sent to the police. 

___________________________________________________________


## This code was build by:
### Aditya amarnath
### Bhargav Adhvaryu


In [ ]:
#First Import the necessary modules
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import datetime
import math
import time
from scipy.io.wavfile import write


#Declare the camera to capture videos
cap =cv2.VideoCapture(0)
car_cascade = cv2.CascadeClassifier('vech.xml')#cv2.data.haarcascades+ 'haarcascade_frontalface_default.xml'
frame_size = (int(cap.get(3)), int(cap.get(4)))

#Declare the variables necessary for recording
detection = False
SECONDS_TO_RECORD_AFTER_DETECTION = 1
timer_started = False
detection_stopped_time = None

#Our Video Writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

#This is repeated to declare variable out
current_time=datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
out = cv2.VideoWriter(f'{current_time}.mp4', fourcc, 20,frame_size)

while(True):
    #Necessary for taking the video through camera
    _,frame = cap.read()
    

    #Convert it into grayscale

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    #Our actual car detector
    car = car_cascade.detectMultiScale(gray,1.1,5)
    if len(car) >0:
        if detection :
            timer_started = False #To keep a timer ready and start it when the body is not in detection
            
        else: # if we detect 1 or more body
            detection = True

            #Here we will start recording
            current_time=datetime.datetime.now().strftime('%d-%m-%Y-%H-%M-%S')
            out = cv2.VideoWriter(f'{current_time}.mp4', fourcc, 20,frame_size)
            print('Started Recording!!')
            print({current_time})
            #This is the email system for the police
            
            
    #To stop the recording when the body is out of detection for more than 5 seconds
    elif detection:
        if timer_started:
            if time.time() - detection_stopped_time >= SECONDS_TO_RECORD_AFTER_DETECTION:
                detection = False
                timer_started = False
                
                print('Stop recording!!')
            
        else:
        #Here if the body comes back into detection before 1 seconds then the recording is continued
            timer_started =True
            detection_stopped_time = time.time() 
    for (x,y,width,height) in car:
        cv2.rectangle(frame,(x,y),(x+width ,x+height),(0,0,255),2)
    out.write(frame)
    cv2.imshow('Camera',frame)  
    
        
    if cv2.waitKey(1) == ord('q'):  
        break

out.release()
cap.release()                       
cv2.destroyAllWindows()

#### This code below is being run with the video taken above , but if used as a live system then we don't use the video recording system above. 

#### However this way of using this code is still in development and still needs improvement. 
#### Hence, we apologize for any errors the user may recieve.


In [ ]:
import cv2
import dlib
import time
import math
import smtplib
import ssl
from email.message import EmailMessage

#The car detector

carCascade = cv2.CascadeClassifier('vech.xml')

#This is where you can put your video file
#IF it is a live system then make the below code as video = cv2.VideoCapture(0) where 0 means that it is accessing our camera.

video = cv2.VideoCapture('Video/traffic-27260.mp4')

#Needed for the window size
WIDTH = 1280
HEIGHT = 720



speedfinal =[]

#This is necessary for calculating the speed
def estimateSpeed(location1, location2):
    d_pixels = math.sqrt(math.pow(location2[0] - location1[0], 2) + math.pow(location2[1] - location1[1], 2))
    #This is an alternate option other than using the ppm which we have selected which is more optimal.
    #ppm= location2[2] / carWidth.
    #carWidth  = YOUR CHOICE
    ppm = 8.8
    d_meters = d_pixels / ppm
    
    fps = 18
    speed = d_meters * fps * 3.6
    return speed

#This is necessary to create tracking ID's and then track different vehicles to detect their speed
def trackMultipleObjects():
    rectangleColor = (0, 255, 0)
    frameCounter = 0
    currentCarID = 0
    fps = 0

    carTracker = {}
    carNumbers = {}
    carLocation1 = {}
    carLocation2 = {}
    speed = [None] * 1000
    
    #This is needed to create a video

    out = cv2.VideoWriter('The_Traffic_Video.avi', cv2.VideoWriter_fourcc('M','J','P','G'), 10, (WIDTH, HEIGHT))

    while True:
        start_time = time.time()
        rc, image = video.read()
        if type(image) == type(None):
            break
        #Gaussian = cv2.GaussianBlur(image,ksize=(15,15))
        image = cv2.resize(image, (WIDTH, HEIGHT))
        
        resultImage = image.copy()
        frameCounter = frameCounter + 1
        carIDtoDelete = []

        for carID in carTracker.keys():
            trackingQuality = carTracker[carID].update(image)

            if trackingQuality < 7:
                carIDtoDelete.append(carID)

        #This is to remove the carID as soon as the vehicle is not detected which is to say that it leaves
        for carID in carIDtoDelete:
            print("Removing carID " + str(carID) + ' from list of trackers. ')
            print("Removing carID " + str(carID) + ' previous location. ')
            print("Removing carID " + str(carID) + ' current location. ')
            carTracker.pop(carID, None)
            carLocation1.pop(carID, None)
            carLocation2.pop(carID, None)

        
        if not (frameCounter % 10):
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            cars = carCascade.detectMultiScale(gray, 1.1, 13, 18, (24, 24))

            for (_x, _y, _w, _h) in cars:
                x = int(_x)
                y = int(_y)
                w = int(_w)
                h = int(_h)

                x_bar = x + 0.5 * w
                y_bar = y + 0.5 * h

                matchCarID = None
                
                
        #From here begins our tracker system

                for carID in carTracker.keys():
                    trackedPosition = carTracker[carID].get_position()

                    t_x = int(trackedPosition.left())
                    t_y = int(trackedPosition.top())
                    t_w = int(trackedPosition.width())
                    t_h = int(trackedPosition.height())

                    t_x_bar = t_x + 0.5 * t_w
                    t_y_bar = t_y + 0.5 * t_h

                    if ((t_x <= x_bar <= (t_x + t_w)) and (t_y <= y_bar <= (t_y + t_h)) and (x <= t_x_bar <= (x + w)) and (y <= t_y_bar <= (y + h))):
                        matchCarID = carID

                if matchCarID is None:
                    print(' Creating new tracker' + str(currentCarID))

                    tracker = dlib.correlation_tracker()
                    tracker.start_track(image, dlib.rectangle(x, y, x + w, y + h))

                    carTracker[currentCarID] = tracker
                    carLocation1[currentCarID] = [x, y, w, h]

                    currentCarID = currentCarID + 1

        for carID in carTracker.keys():
            trackedPosition = carTracker[carID].get_position()

            t_x = int(trackedPosition.left())
            t_y = int(trackedPosition.top())
            t_w = int(trackedPosition.width())
            t_h = int(trackedPosition.height())

            cv2.rectangle(resultImage, (t_x, t_y), (t_x + t_w, t_y + t_h), rectangleColor, 4)

            carLocation2[carID] = [t_x, t_y, t_w, t_h]

        end_time = time.time()

        if not (end_time == start_time):
            fps = 1.0/(end_time - start_time)

        for i in carLocation1.keys():
            if frameCounter % 1 == 0:
                [x1, y1, w1, h1] = carLocation1[i]
                [x2, y2, w2, h2] = carLocation2[i]

                carLocation1[i] = [x2, y2, w2, h2]

                if [x1, y1, w1, h1] != [x2, y2, w2, h2]:
                    if (speed[i] == None or speed[i] == 0) and y1 >= 275 and y1 <= 285:
                        speed[i] = estimateSpeed([x1, y1, w1, h1], [x1, y2, w2, h2])

                    if speed[i] != None and y1 >= 180:
                        cv2.putText(resultImage, str(int(speed[i])) + "Km/h", (int(x1 + w1/2), int(y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 100) ,2)
                        #cv2.putText(resultImage, str(currentCarID), (int(x1 + w1/2), int(y1-20)), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0) ,2)
                        if speed[i]>90:
                            

                            
                            speedfinal.append(str(int(speed[i])))
                            print(speedfinal)
                            print('Car no. ', currentCarID, 'is overspeeding')
                            
                             
                        else:
                            print('Vehicle no. ', currentCarID , 'is not over speeding')
                            
        #Finally we have to show this to the user                      
                                           
                        
        cv2.imshow('result', resultImage)
        cv2.imshow('roi',resultImage[300:700,:])
        
        #We have to finally save the video
        out.write(resultImage)
        
            
        if cv2.waitKey(1) == 27:
            break
        out.release()
    
    cv2.destroyAllWindows()


#We now have to execute the above function and get the desired output
if __name__ == '__main__':
    trackMultipleObjects()
if len(speedfinal)>0:
        
    # Fill out the email details before running the code!!

        email_sender = 'write-email-here'
        email_password = 'write-App - password-here'#For app password check below
        email_receiver = 'write-email-receiver-here'

        # Set the subject and body of the email
        subject = 'Overspeeding Vehicle Found'
        body = """
        Hello overspeeding has been detected
        """

    

        em = EmailMessage()
        em['From'] =  'EMAIL OF SENDER'
        em['To'] = 'EMAIL OF RECEIVER'
        em['Subject'] = subject
        em.set_content(body)

        # Add SSL (layer of security)
        context = ssl.create_default_context()

        # Log in and send the email
        with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:

            smtp.login('EMAIL OF SENDER', 'APP PASSWORD')

            #Here the app password refers to a password which has been created by a system and not by the user.
            #For generating an app password:- 
            #1. Go to your google account(sender email) and enable 2 factor authentification
            #2. You may then search in the google bar above as app password and then click the first suggestion
            #3.Login again and then under select app choose mail and under device select other and give it your own name
            #4.Then click generate and your app password is ready!!

            smtp.sendmail('EMAIL OF SENDER', 'EMAIL OF POLICE(RECEIVER)', em.as_string())
        
        print('Mail has been sent')
